### 🛠️ 1. Initialize notebook variables

Configures everything that's needed for deployment. 

❗️ **Modify entries under _1) User-defined parameters_**.

In [ ]:
import utils
from apimtypes import *

# 1) User-defined parameters (change these as needed)
rg_location = 'eastus2'
index       = 1
apim_sku    = APIM_SKU.BASICV2
deployment  = INFRASTRUCTURE.APIM_ACA

# 2) Service-defined parameters (please do not change these)
rg_name = utils.get_infra_rg_name(deployment, index)
rg_tags = utils.build_infrastructure_tags(deployment)

# 3) Define the APIs and their operations and policies

# Policies
hello_world_policy_xml      = utils.read_policy_xml(HELLO_WORLD_XML_POLICY_PATH)
backend_polixy_xml          = utils.read_policy_xml(BACKEND_XML_POLICY_PATH)
aca_backend_1_policy_xml    = backend_polixy_xml.format(backend_id = 'aca-backend-1')
aca_backend_2_policy_xml    = backend_polixy_xml.format(backend_id = 'aca-backend-2')
aca_backend_pool_policy_xml = backend_polixy_xml.format(backend_id = 'aca-backend-pool')

# Hello World (Root)
api_hwroot_get = GET_APIOperation('This is a GET for Hello World in the root', hello_world_policy_xml)
api_hwroot  = API('hello-world', 'Hello World', '', 'This is the root API for Hello World', operations = [api_hwroot_get])

# Hello World (ACA Backend 1)
api_hwaca_1_get = GET_APIOperation('This is a GET for Hello World on ACA Backend 1')
api_hwaca_1 = API('hello-world-aca-1', 'Hello World (ACA 1)', '/aca-1', 'This is the ACA API for Backend 1', policyXml = aca_backend_1_policy_xml, operations = [api_hwaca_1_get])

# Hello World (ACA Backend 2)
api_hwaca_2_get = GET_APIOperation('This is a GET for Hello World on ACA Backend 2')
api_hwaca_2 = API('hello-world-aca-2', 'Hello World (ACA 2)', '/aca-2', 'This is the ACA API for Backend 2', policyXml = aca_backend_2_policy_xml, operations = [api_hwaca_2_get])

# Hello World (ACA Backend Pool)
api_hwaca_pool_get = GET_APIOperation('This is a GET for Hello World on ACA Backend Pool')
api_hwaca_pool = API('hello-world-aca-pool', 'Hello World (ACA Pool)', '/aca-pool', 'This is the ACA API for Backend Pool', policyXml = aca_backend_pool_policy_xml, operations = [api_hwaca_pool_get])

# APIs Array
apis: List[API] = [api_hwroot, api_hwaca_1, api_hwaca_2, api_hwaca_pool]

utils.print_ok('Notebook initialized')

### 🚀 2. Create deployment using Bicep

Creates the bicep deployment into the previously-specified resource group. A bicep parameters file will be created prior to execution.

In [ ]:
import utils

# 1) Define the Bicep parameters with serialized APIs
bicep_parameters = {
    'apimSku'   : {'value': apim_sku.value},
    'apis'      : {'value': [api.to_dict() for api in apis]}
}

# 2) Run the deployment
output = utils.create_bicep_deployment_group(rg_name, rg_location, deployment, bicep_parameters, rg_tags = rg_tags)

# 3) Check the deployment outputs
if not output.success:
    raise SystemExit('Deployment failed')

if output.success and output.json_data:
    apim_gateway_url    = output.get('apimResourceGatewayURL', 'APIM API Gateway URL')
    aca_url_1           = output.get('acaUrl1', 'ACA Backend 1 URL')
    aca_url_2           = output.get('acaUrl2', 'ACA Backend 2 URL')

utils.print_ok('Deployment completed')

### ✅ 3. Verify API Request Success

Assert that the deployment was successful by making simple calls to APIM. 

In [ ]:
import utils
from apimrequests import ApimRequests
from apimtesting import ApimTesting

reqs = ApimRequests(apim_gateway_url)
tests = ApimTesting("APIM-ACA Tests")

output = reqs.singleGet('/', msg = 'Calling Hello World (Root) API')
tests.verify(output, 'Hello World from API Management!')

output = reqs.singleGet('/aca-1/', msg = 'Calling Hello World (ACA Backend 1) API')
tests.verify(output, 'Hello World!')

output = reqs.singleGet('/aca-2/', msg = 'Calling Hello World (ACA Backend 2) API')
tests.verify(output, 'Hello World!')

output = reqs.multiGet('/aca-pool/', 3, msg = 'Calling Hello World (ACA Backend Pool) API')
tests.verify(len(output), 3)
tests.verify(output[0]['response'], 'Hello World!')
tests.verify(output[1]['response'], 'Hello World!')
tests.verify(output[2]['response'], 'Hello World!')

tests.print_summary()

utils.print_ok('All done!')

### 🗑️ Clean up resources

When you're finished experimenting, it's advisable to remove all associated resources from Azure to avoid unnecessary cost.
Use the [clean-up notebook](clean-up.ipynb) for that.